In [1]:
import numpy as np
import pandas as pd
import modin.pandas as mpd
import os
import ray

In [2]:
os.environ["MODIN_ENGINE"] = "ray"

runtime_env = {
    'env_vars': {
        "RAY_memory_monitor_refresh_ms": "100",
        "RAY_memory_usage_threshold": "0.85"
     }
}
ray.init(runtime_env=runtime_env)

2024-04-13 09:46:31,038	INFO worker.py:1743 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.10.12
Ray version:,2.10.0
Dashboard:,http://127.0.0.1:8265


In [4]:
api = mpd.read_csv("csv/APICALLS-f2d849a0-e5ff-45d0-a98e-c8491bd1a44a-full.csv")

Data types of partitions are different! Please refer to the troubleshooting section of the Modin documentation to fix this issue.


In [5]:
api.shape

(40000, 63466)

In [8]:
api.head()

,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,android.view.View.performAccessibilityAction,android.widget.TextView.getCompoundPaddingTop,android.app.NotificationManager.getNotificationChannels,android.view.View.setTransitionAlpha,android.view.View.getWindowVisibleDisplayFrame,...,android.widget.ZoomControls.getTop,android.widget.ZoomControls.getLeft,java.nio.file.attribute.PosixFilePermissions.toString,android.view.ViewGroup.hasOnLongClickListeners,android.widget.ProgressBar.setForegroundTintList,android.widget.Toolbar.getParent,android.widget.Toolbar.getBottom,android.graphics.drawable.AnimatedVectorDrawable.mutate,android.widget.TextClock.onMeasure,android.widget.TextClock.setShadowLayer
0,0,0,0,0,0,1.0,1.0,2.0,2.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1,1,1,0,2.0,0.0,2.0,2.0,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2,2,2,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,3,3,3,0,0.0,0.0,0.0,1.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,4,4,4,0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
api = api.fillna(0)

In [10]:
api = api.drop(["Unnamed: 0.4", "Unnamed: 0.3", "Unnamed: 0.2", "Unnamed: 0.1", "Unnamed: 0"], axis=1)

In [11]:
api

,android.view.View.performAccessibilityAction,android.widget.TextView.getCompoundPaddingTop,android.app.NotificationManager.getNotificationChannels,android.view.View.setTransitionAlpha,android.view.View.getWindowVisibleDisplayFrame,android.app.SharedElementCallback.onCaptureSharedElementSnapshot,java.util.Collections.synchronizedList,android.widget.EditText.onDragEvent,android.app.SearchableInfo.getSearchActivity,android.widget.ImageButton.getDrawable,...,android.widget.ZoomControls.getTop,android.widget.ZoomControls.getLeft,java.nio.file.attribute.PosixFilePermissions.toString,android.view.ViewGroup.hasOnLongClickListeners,android.widget.ProgressBar.setForegroundTintList,android.widget.Toolbar.getParent,android.widget.Toolbar.getBottom,android.graphics.drawable.AnimatedVectorDrawable.mutate,android.widget.TextClock.onMeasure,android.widget.TextClock.setShadowLayer
0,1.0,1.0,2.0,2.0,5.0,1.0,3.0,1.0,4.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.0,0.0,2.0,2.0,8.0,0.0,3.0,1.0,4.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,5.0,0.0,12.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,4.0,0.0,18.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
39996,0.0,0.0,0.0,2.0,7.0,0.0,17.0,0.0,4.0,18.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
39997,3.0,0.0,0.0,2.0,0.0,0.0,17.0,0.0,4.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
39998,1.0,4.0,0.0,1.0,7.0,0.0,4.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
print(f"Goodware class len {len(api[api['class']==0])}; Malware class len {len(api[api['class']==1])}")

Goodware class len 20000; Malware class len 20000


In [14]:
api['file_name']

KeyError: 'file_name'

In [21]:
y = np.array(api["class"])

In [16]:
y

array([0, 1, 1, ..., 0, 0, 1])

In [25]:
api = api.drop('class', axis=1)

In [26]:
api.shape

(40000, 63460)

In [27]:
x = api.to_numpy(na_value=0)

(raylet) Spilled 2271 MiB, 30 objects, write throughput 353 MiB/s. Set RAY_verbose_spill_logs=0 to disable this message.
(raylet) Spilled 4539 MiB, 60 objects, write throughput 461 MiB/s.
(raylet) Spilled 8544 MiB, 113 objects, write throughput 128 MiB/s.
(raylet) Spilled 39874 MiB, 530 objects, write throughput 117 MiB/s.


In [28]:
x

array([[1., 1., 2., ..., 0., 0., 0.],
       [2., 0., 2., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [3., 0., 0., ..., 0., 0., 0.],
       [1., 4., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [29]:
with open("arrays/api-x-full-ordered.npy", "wb") as file:
    np.save(file, x)

In [30]:
with open("arrays/api-y-full-ordered.npy", "wb") as file:
    np.save(file, y)

In [37]:
x = np.load(open("arrays/apicalls-x-full-ordered.npy", "rb"))

In [38]:
y = np.load(open("arrays/apicalls-y-full-ordered.npy", "rb"))

In [39]:
x

array([[1., 1., 2., ..., 0., 0., 0.],
       [2., 0., 2., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [3., 0., 0., ..., 0., 0., 0.],
       [1., 4., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [40]:
y

array([0, 1, 1, ..., 0, 0, 1])

In [41]:
from sklearn.decomposition import PCA
pca = PCA(n_components=100, random_state=42)

In [42]:
x.shape

(40000, 63460)

In [43]:
x_pca = pca.fit_transform(x)

In [44]:
x_pca.shape

(40000, 100)

In [45]:
x_pca

array([[-1.70880280e+04, -1.42851608e+02,  3.64254787e+02, ...,
         4.54390590e+01,  1.52617087e+01,  6.75148374e+01],
       [-7.26246953e+03, -5.46679242e+02, -1.94783863e+03, ...,
         7.04471493e+01, -7.20040779e+01, -6.21567836e+01],
       [-3.41528143e+04, -1.26732343e+02, -2.10138066e+03, ...,
        -1.85849548e+01, -4.36247553e+01,  8.88083937e+00],
       ...,
       [ 4.35124449e+04, -9.61686427e+02,  6.56266692e+03, ...,
        -1.09087275e+02, -4.59518413e+01, -1.33488920e+02],
       [-1.86966828e+04,  1.77059657e+01,  2.64112165e+03, ...,
        -2.46206285e+01,  2.16381078e+01,  6.12275622e+01],
       [-2.08891412e+04, -1.94148703e+02,  1.31975225e+03, ...,
         7.93445233e+01,  4.00207670e+01,  5.92949236e+01]])

In [36]:
with open("arrays/apicalls-x-pca-ordered.npy", "wb") as file:
    np.save(file, x_pca)